In [148]:
import json
import spacy
import nltk
import numpy as np
from tqdm import tqdm, tqdm_notebook
from collections import Counter
import matplotlib.pyplot as plt
import pandas as pd
import gensim
import seaborn as sns
from xgboost.sklearn import XGBClassifier 
from nltk.tokenize import word_tokenize
from sklearn.model_selection import train_test_split
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
import random
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import warnings
warnings.filterwarnings('ignore')

In [2]:
with open('relevant_news_10K.json',encoding="utf8") as f1:
    data1 = json.load(f1)
    
with open('irrelevant_news_10K.json', encoding = "utf8") as f2:
    data2 = json.load(f2)

In [4]:
headlines = []
summaries = []
sources = []
content = []

for d in data1:
    headlines.append(d['headline'])
    summaries.append(d['summary'])
    sources.append(d['source'])
    content.append(d['content'])
    
for d in data2:
    headlines.append(d['headline'])
    summaries.append(d['summary'])
    sources.append(d['source'])
    content.append(d['content'])

In [10]:
len(content)

20000

In [11]:
label = [[1]]*len(data1)
label.extend([[0]]*len(data2))


In [17]:
# Let's use content as our data
from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(content, label, test_size=0.20, random_state=42, shuffle = True)

In [22]:
def read_corpus(corpus, tokens_only=False):
    #with smart_open.smart_open(fname, encoding="iso-8859-1") as f:
    for i, line in tqdm_notebook(enumerate(corpus), total = len(corpus)):
        if tokens_only:
            yield gensim.utils.simple_preprocess(line)
        else:
            # For training data, add tags
            yield gensim.models.doc2vec.TaggedDocument(gensim.utils.simple_preprocess(line), [i])

In [23]:
%%time
train_corpus = list(read_corpus(X_train))
test_corpus = list(read_corpus(X_test, tokens_only=True))


Wall time: 1min 3s


In [79]:
# Tune parameters <TODO>
model = gensim.models.doc2vec.Doc2Vec(vector_size=100, min_count=2, epochs=10, workers= 4,compute_loss = True)

In [80]:
# try different vocab builds <TODO>
model.build_vocab(train_corpus)

In [81]:
class EpochLogger(CallbackAny2Vec):
    
    def __init__(self):
        self.epoch = 0

    def on_epoch_begin(self, model):
             print("Epoch #{} start".format(self.epoch))

    def on_epoch_end(self, model):
            print("Epoch #{} end".format(self.epoch))
            #print("Loss #{}".format(model.get_latest_training_loss()))
            self.epoch += 1

In [82]:
%time 
from gensim.models.callbacks import CallbackAny2Vec
epoch_logger = EpochLogger()
model.train(train_corpus,  total_examples=model.corpus_count,epochs=model.epochs, callbacks=[epoch_logger])
#model.get_latest_training_loss()

Wall time: 0 ns
Epoch #0 start
Epoch #0 end
Epoch #1 start
Epoch #1 end
Epoch #2 start
Epoch #2 end
Epoch #3 start
Epoch #3 end
Epoch #4 start
Epoch #4 end
Epoch #5 start
Epoch #5 end
Epoch #6 start
Epoch #6 end
Epoch #7 start
Epoch #7 end
Epoch #8 start
Epoch #8 end
Epoch #9 start
Epoch #9 end


In [43]:
model.wv.vocab['oil'].count

2094

In [84]:
ranks = []
second_ranks = []
for doc_id in tqdm_notebook(range(len(train_corpus))):
    inferred_vector = model.infer_vector(train_corpus[doc_id].words)
    sims = model.docvecs.most_similar([inferred_vector], topn=len(model.docvecs))
    rank = [docid for docid, sim in sims].index(doc_id)
    ranks.append(rank)
    
    second_ranks.append(sims[1])

In [88]:
Counter(ranks)   #93% of the time found to be similar to the same document

Counter({0: 14963,
         1: 882,
         7: 6,
         6: 10,
         4: 22,
         48: 1,
         14: 3,
         2: 41,
         15: 3,
         3: 20,
         5: 13,
         18: 2,
         13: 5,
         11: 2,
         31: 1,
         12: 4,
         10: 2,
         8: 5,
         34: 1,
         23: 2,
         9: 1,
         17: 3,
         20: 1,
         627: 1,
         786: 1,
         16: 1,
         71: 1,
         168: 1,
         19: 1,
         27: 1})

In [97]:
# Pick a random document from the test corpus and infer a vector from the model
doc_id = random.randint(0, len(test_corpus))
inferred_vector = model.infer_vector(test_corpus[1876])  #doc_id
sims = model.docvecs.most_similar([inferred_vector], topn=len(model.docvecs))

# Compare and print the most/median/least similar documents from the train corpus
print('Test Document ({}): «{}»\n'.format(doc_id, ' '.join(test_corpus[doc_id])))
print(u'SIMILAR/DISSIMILAR DOCS PER MODEL %s:\n' % model)
for label, index in [('MOST', 0), ('MEDIAN', len(sims)//2), ('LEAST', len(sims) - 1)]:
    print(u'%s %s: «%s»\n' % (label, sims[index], ' '.join(train_corpus[sims[index][0]].words)))

Test Document (2238): «may corelogic will close its bloomington office and up to employees will be laid off the irvine calif based provider of real estate property information analytics and data services filed notice with the minnesota department of employment and economic development that its office at th ave will close by dec layoffs will begin july the notice was required under the worker adjustment and retraining notification act the bloomington office is part of the company appraisal operation some of the work will be transitioned to location in dallas and workers may apply for jobs there said company spokeswoman alyson austin via mail corelogic said in december that it would be accelerating its multiyear restructuring plan that includes the appraisal management operations the publicly traded company had total employees as of dec of them in the united states according to its most recent annual report with the securities and exchange commission corelogic has properties in states an

In [ ]:
'''
model.save('doc2vec_embedding_1.model')   #memory error !! <TODO> optimization 
'''

## Create feature vectors from Doc2vec embeddings

In [123]:
#Doc2vec feature vectors
X_train = np.array([model.docvecs[i] for i in range(len(train_corpus))])
X_test = np.array([model.infer_vector(test_corpus[i]) for i in range(len(test_corpus))])

## XGBoost Classifier

In [137]:
xclas = XGBClassifier()  # and for classifier  
xclas.fit(X_train, np.array(Y_train))
y_pred = xclas.predict(X_test)  

In [138]:
print(classification_report(Y_test,y_pred))

              precision    recall  f1-score   support

           0       0.69      0.60      0.64      1981
           1       0.65      0.73      0.69      2019

    accuracy                           0.67      4000
   macro avg       0.67      0.67      0.67      4000
weighted avg       0.67      0.67      0.67      4000



## Confusion Matrix

In [140]:
print(confusion_matrix(Y_test,y_pred))

[[1198  783]
 [ 540 1479]]


In [141]:
print(accuracy_score(Y_test,y_pred))

0.66925


In [143]:
#Sanity Check: test on train data
#Expected Result: Should overfit
xclas.fit(X_train, np.array(Y_train))
y_pred_1 = xclas.predict(X_train)
print(accuracy_score(np.array(Y_train),y_pred_1))

0.721375


### Test Data: 67% Train Data: 72% -> Model has not learnt enough!

## Naive Bayes Classifier

In [149]:
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()
y_pred_gnb = gnb.fit(X_train, np.array(Y_train)).predict(X_test)
print(accuracy_score(np.array(Y_test),y_pred_gnb))

0.61175


In [150]:
y_pred_gnb_1 = gnb.fit(X_train, np.array(Y_train)).predict(X_train)
print(accuracy_score(np.array(Y_train),y_pred_gnb_1))

0.6005625


### Test Data: 60% Train Data: 61%

## Logistic Regression

In [151]:
logistic = LogisticRegression()
y_pred_lgr = logistic.fit(X_train, np.array(Y_train)).predict(X_test)
print(accuracy_score(np.array(Y_test),y_pred_lgr))

0.65825


In [152]:
y_pred_lgr_1 = logistic.fit(X_train, np.array(Y_train)).predict(X_train)
print(accuracy_score(np.array(Y_train),y_pred_lgr_1))

0.6656875


### Test Data: 65% Train Data: 66%

*TODOS  
* Pretty low accuracy, features not representative 
* need to fine-tune embedding model and re-train with different parameters
* Experiment with different embedding models
* Performance of Word2vec > Doc2vec, Try mean(word2vec) of each document
* Unable to save embedding_model, reduce feature vector_size?
* Try other classifiers and other feature sets (tf-idf, bow)
*  Current model can be used for explainability model training experiments
